In [1]:

from certified_iris_generator import CertifiedIrisRegionGenerator
import sys
import os
import time
import numpy as np
from functools import partial
import itertools
import mcubes
import visualizations_utils as viz_utils
import iris_utils #TODO remove
from iris_plant_visualizer import IrisPlantVisualizer

In [2]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.all import InverseKinematics, RevoluteJoint, RationalForwardKinematics
from pydrake.geometry.optimization import IrisOptionsRationalSpace, IrisInRationalConfigurationSpace, HPolyhedron, Hyperellipsoid
import pydrake.symbolic as sym
from pydrake.all import MathematicalProgram
import meshcat

# Build plant

In [3]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
parser.package_map().Add( "wsg_50_description", os.path.dirname(FindResourceOrThrow(
            "drake/manipulation/models/wsg_50_description/package.xml")))

simple_collision = True
directives_file = FindResourceOrThrow("drake/sos_iris_certifier/planar_iiwa_simple_collision_welded_gripper.yaml") \
    if simple_collision else FindResourceOrThrow("drake/sos_iris_certifier/planar_iiwa_dense_collision_welded_gripper.yaml")
directives = LoadModelDirectives(directives_file)
models = ProcessModelDirectives(directives, plant, parser)

q0 = [-0.2, -1.2, 1.6]
index = 0
for joint_index in plant.GetJointIndices(models[0].model_instance):
    joint = plant.get_mutable_joint(joint_index)
    if isinstance(joint, RevoluteJoint):
        joint.set_default_angle(q0[index])
        index += 1

plant.Finalize()


# Setup meshcat visualization

In [4]:
do_viz = True
visualizer = IrisPlantVisualizer(plant, builder, scene_graph)
diagram = visualizer.diagram

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Connected to meshcat-server.


# Set up IRIS options

In [5]:
iris_options = IrisOptionsRationalSpace()
iris_options.require_sample_point_is_contained = True
iris_options.iteration_limit = 5
iris_options.configuration_space_margin = 1e-5
iris_options.termination_threshold = -1
iris_options.relative_termination_threshold = 0.05
iris_options.enable_ibex = False
iris_options.certify_region_with_sos_during_generation = False
iris_options.certify_region_with_sos_after_generation = False
# uncomment to test if non-default is working
# currently throwing std::bad_alloc if we try to set q_star
# iris_options.q_star = 0.5 * np.ones(3)


#seed points specified in q space
seed_points = np.array([
                        [0.0, -2.016, 1.975], # in tight
                        [-1, -2, 0.5],        # neutral pose
                        [0.3, -0.8, 0.5],     # above shelf
                        [0.25, -1.6, -0.25],  # in shelf 1
                        [0.07, -1.8, -0.2],   # leaving shelf 1
                        [-0.1, -2, -0.3]      # out of shelf 1
                        ])


In [6]:
regions = []
context = diagram.CreateDefaultContext()
for i, s in enumerate(seed_points):
    plant.SetPositions(plant.GetMyMutableContextFromRoot(context), s)
    if i > 0:
        starting_hpolyhedron = regions[i-1]
        r = IrisInRationalConfigurationSpace (plant, plant.GetMyContextFromRoot(context),
                                              iris_options, starting_hpolyhedron)
    else:
        r = IrisInRationalConfigurationSpace(plant, plant.GetMyContextFromRoot(context), iris_options)
    print(f"{r.A()}t <= {r.b()}")
    regions.append(r)
    print(f'Completed region: {i+1}/{len(seed_points)}')

[[ 1.          0.          0.        ]
 [ 0.          1.          0.        ]
 [ 0.          0.          1.        ]
 [-1.         -0.         -0.        ]
 [-0.         -1.         -0.        ]
 [-0.         -0.         -1.        ]
 [ 0.1456932   0.97734169 -0.1535471 ]
 [ 0.14507765  0.97918606 -0.14194059]
 [ 0.13324709  0.9901976   0.04187985]
 [ 0.13325018  0.99019891  0.04183915]
 [ 0.13325325  0.99020021  0.04179845]
 [ 0.13325633  0.99020152  0.04175774]
 [ 0.13325941  0.99020282  0.04171703]
 [ 0.17673237  0.97152073 -0.15783896]
 [ 0.17723675  0.96946325 -0.16949377]
 [ 0.17721125  0.96957301 -0.16889152]
 [ 0.17718429  0.9696884  -0.16825618]
 [ 0.17715561  0.96981042 -0.16758175]
 [ 0.13834134  0.69751086 -0.70309336]
 [ 0.13851139  0.7114772  -0.68892292]
 [ 0.1375092   0.85533313 -0.4994962 ]
 [ 0.13750956  0.85531611 -0.49952525]
 [ 0.16696619  0.67944661 -0.71447505]
 [ 0.16685319  0.69267722 -0.70168247]
 [ 0.1622871   0.83192949 -0.53061871]
 [ 0.16228847  0.83190796

In [7]:
# plot regions and collision constraint
if do_viz:
    visualizer.plot_regions(regions)
    visualizer.plot_seedpoints(np.tan(seed_points/2))
    visualizer.visualize_collision_constraint(N = 50)

In [8]:
rational_forward_kinematics = RationalForwardKinematics(plant)
q_star = iris_options.q_star if iris_options.q_star is not None else np.zeros(plant.num_positions())

In [9]:
starting_hpolyhedron = HPolyhedron(np.array([[0,1,0]]), np.array([-0.3]))
t_lower_limits = rational_forward_kinematics.ComputeTValue(plant.GetPositionLowerLimits(), q_star)
t_upper_limits = rational_forward_kinematics.ComputeTValue(plant.GetPositionUpperLimits(), q_star)
joint_limits = HPolyhedron.MakeBox(t_lower_limits, t_upper_limits)

plant.SetPositions(plant.GetMyMutableContextFromRoot(context), seed_points[2])
r = IrisInRationalConfigurationSpace(plant, plant.GetMyContextFromRoot(context),
                                              iris_options, starting_hpolyhedron)

starting_hpolyhedron_viz = starting_hpolyhedron.IrredundantIntersection(joint_limits)
regions2 = [r, starting_hpolyhedron_viz]
if do_viz:
    visualizer.plot_regions(regions2, region_suffix = '2')